In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [ ]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = {}):
    if(logs.get("accuracy") >= DESIRED_ACCURACY):
      print("Reached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

--2021-02-23 05:17:22--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.197.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.03s   

2021-02-23 05:17:22 (99.7 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [ ]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation = "relu", input_shape = (150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = "relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation = "relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = "relu"),
    tf.keras.layers.Dense(1, activation = "sigmoid")
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss = "binary_crossentropy", optimizer = RMSprop(lr = .001), metrics = ["accuracy"])

In [ ]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1 / 255)

train_generator = train_datagen.flow_from_directory(
        "/tmp/h-or-s", target_size = (150, 150), batch_size = 10, class_mode = "binary")

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [ ]:
# This code block should call model.fit and train for
# a number of epochs. 
#history = model.fit(
#      train_generator, training_labels, epochs = 1000, callbacks = [callbacks])
history = model.fit_generator(
      train_generator, steps_per_epoch = 1, epochs = 1000, verbose = 1, callbacks = [callbacks])

# Expected output: "Reached 99.9% accuracy so cancelling training!""

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/1000
1/1 [==============================] - 1s 1s/step - loss: 0.7074 - accuracy: 0.3000
Epoch 2/1000
1/1 [==============================] - 0s 285ms/step - loss: 12.8471 - accuracy: 0.3000
Epoch 3/1000
1/1 [==============================] - 0s 289ms/step - loss: 1.1308 - accuracy: 0.5000
Epoch 4/1000
1/1 [==============================] - 0s 293ms/step - loss: 0.6548 - accuracy: 0.5000
Epoch 5/1000
1/1 [==============================] - 0s 275ms/step - loss: 0.5845 - accuracy: 0.9000
Epoch 6/1000
1/1 [==============================] - 0s 281ms/step - loss: 0.9794 - accuracy: 0.4000
Epoch 7/1000
1/1 [==============================] - 0s 290ms/step - loss: 0.5912 - accuracy: 0.9000
Epoch 8/1000
1/1 [==============================] - 0s 291ms/step - loss: 0.4701 - accuracy: 0.7000
Epoch 9/1000
1/1 [==============================] - 0s 283ms/step - loss: 0.4743 - accuracy: 0.8000
Epoch 10/1000
1/1 [==============================] - 0s 302ms/step - loss: 0.6788 - accuracy: 0.4000
E